In [5]:
from config_parsing import parse_config
import plotly.io as pio

%load_ext autoreload
%autoreload 2

pio.renderers.default = "jupyterlab"
 
CONFIG_LOCATION = "./Docs.json"
config = parse_config(CONFIG_LOCATION)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
list(config.machines.keys())

['Build_OilRefinery_C',
 'Build_FoundryMk1_C',
 'Build_Packager_C',
 'Build_Blender_C',
 'Build_ManufacturerMk1_C',
 'Build_AssemblerMk1_C',
 'Build_SmelterMk1_C',
 'Build_ConstructorMk1_C',
 'Build_HadronCollider_C',
 'Build_OilPump_C',
 'Build_MinerMk2_C',
 'Build_MinerMk3_C',
 'Build_MinerMk1_C',
 'Build_WaterPump_C',
 'Build_GeneratorCoal_C',
 'Build_GeneratorFuel_C',
 'Build_GeneratorBiomass_C',
 'Build_GeneratorNuclear_C',
 'Build_GeneratorGeoThermal_C']

In [10]:
from material import MaterialSpec
from plotting.tables import recipe_summary
from plotting.sankey import plot_sankey
from plotting.graph import plot_process
from process import Process
from machine import Machine
from IPython.display import display, HTML
from igraph import Graph

def factory_plan(available_machines: list[Machine], target: MaterialSpec):
    Machine.autobind(available_machines)

    solution = Process.from_outputs(target,
                                    available_machines
                                    )

    # TODO: color by machine or recipe type
    plot_process(solution, Graph.layout_reingold_tilford)
    plot_sankey(solution)
    display(HTML(recipe_summary(solution)))
    # total_power(solution)
    return solution

In [11]:
# total production
config = parse_config(CONFIG_LOCATION)
available_machines = [m.instance() for m in config.recipes.values()]
target_product = config.materials(iron_rod=10, iron_plate=10, reinforced_iron_plate=5, rotor=5,
                                  steel_beam=5, encased_industrial_beam=2, modular_frame=5, heavy_modular_frame=5)

# factory_plan(available_machines, target_product)

In [10]:
Machine.autobind(available_machines)

target_product = config.materials(iron_plate=1, iron_rod=2)
target_available = config.materials(iron_ore=360)
solution = Process.from_inputs(target_available,
                               target_product,
                               available_machines)

plot_process(solution, Graph.layout_reingold_tilford)
plot_sankey(solution)
display(HTML(recipe_summary(solution)))

AttributeError: 'NoneType' object has no attribute 'round'

In [16]:
from plotting.widgets import material_widget, recipe_widget, optimize_outputs_widget, optimize_inputs_widget
from ipywidgets.widgets import Accordion
from IPython.display import display

a = None

def callback(process):
    global a
    print(process)
    a = process
    

m_w, materials = material_widget(config.materials)
r_w, recipes = recipe_widget(config.recipes)
o_b = optimize_outputs_widget(callback, materials, recipes)

accordion = Accordion(children=[m_w, r_w, o_b])
accordion.set_title(0, 'Materials')
accordion.set_title(1, 'Recipes')
accordion.set_title(2, 'Run')
accordion

Accordion(children=(interactive(children=(IntSlider(value=0, description='uranium_waste', max=1000, min=-1000)…

In [17]:
a

In [12]:
from ipywidgets import widgets
from categorized_collection import CategorizedCollection
from thefuzz import process
from IPython.display import display
from traitlets import Bool

options = config.recipes

fuzz_threshold = .5

search = widgets.Text(
    placeholder='Search Recipes',
    disabled=False,
)

class CustomToggle(widgets.Button):
    value = Bool(False).tag(sync=True)
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.on_click(self.toggle)
    
    def toggle(self, *_, **__):
        self.value = not self.value
        self.button_style = "info" if self.value else ""

category_selectors = {tag: CustomToggle(description=tag) for tag in options.tags}
item_selectors = {name: widgets.Checkbox(value=False, description=name, indent=False) for name in options.keys()}

        
def unselect_category_selectors(change):
    new_value = change["new"]
    if not new_value:
        for tag in options.value_tags(change["owner"].description):
            if (selector := category_selectors[tag]).value:
                selector.toggle()
                        
for selector in item_selectors.values():
    selector.observe(unselect_category_selectors, names="value")
        
def select_by_category(b):
    tag = b.description
    for item in options.tags[tag]:
        item_selectors[item].value = b.value

for selector in category_selectors.values():
    selector.on_click(select_by_category)
    
layout_row_flexwrap = widgets.Layout(flex_flow="row wrap")
layout_col_flexwrap = widgets.Layout(flex_flow="row wrap", height="auto", align_content="flex-start")

checkboxes = widgets.VBox(list(item_selectors.values()), layout=layout_col_flexwrap) 

def filter_results(change):
    new_value = change["new"]
    threshold = 75
    
    if not new_value:
        # TODO: cache this value
        filtered = list(item_selectors.values())
    else:
        filtered = [item_selectors[k] for k, score in process.extract(change["new"], options.keys(), limit=None) if score > threshold]
        
    checkboxes.children = filtered
    
def clear_search(b):
    search.value = ""

clear = widgets.Button(description="clear", button_style="danger", layout=widgets.Layout(width="6em"))
clear.on_click(clear_search)
    
search.observe(filter_results, names="value")
display(widgets.HBox([search, clear]))
display(widgets.HBox(list(category_selectors.values()), layout=layout_row_flexwrap))
display(checkboxes)

In [7]:
config.recipes.tags

defaultdict(set,
            {'mk1': {'crude_oil_mk_1', 'water_mk_1'},
             'Build_OilPump_C': {'crude_oil_mk_1'},
             'extractor': {'crude_oil_mk_1', 'water_mk_1'},
             'Build_WaterPump_C': {'water_mk_1'},
             'generator': {'biomass_burner_biomass',
              'biomass_burner_fabric',
              'biomass_burner_flower_petals',
              'biomass_burner_hatcher_remains',
              'biomass_burner_hog_remains',
              'biomass_burner_leaves',
              'biomass_burner_liquid_biofuel',
              'biomass_burner_mycelia',
              'biomass_burner_packaged_liquid_biofuel',
              'biomass_burner_plasma_spitter_remains',
              'biomass_burner_solid_biofuel',
              'biomass_burner_stinger_remains',
              'biomass_burner_wood',
              'coal_generator_coal',
              'coal_generator_compacted_coal',
              'coal_generator_petroleum_coke',
              'fuel_generator_fuel',
 

In [8]:
a

NameError: name 'a' is not defined

In [ ]:
process_1 >> process_2

In [20]:
1 << 4

16

In [21]:
0.1 << 4

TypeError: unsupported operand type(s) for <<: 'float' and 'int'

In [ ]:
iron_process = input_materials >> iron_rod_factory >> reinforced_iron_plate_factory

steel_inputs = iron_process & coal_process

steel_process = steel_inputs >> steel_factory 